# Lavoro su dataset cleaned (fonte: data/processed/database_cleaned.csv)

Questo notebook usa come sorgente `data/processed/database_cleaned.csv`
Obiettivo: standardizzazione delle variabili categoriche

In [13]:
# --- PARAMS / ENV (eseguire per primo) ---
import os
import pandas as pd

NOTEBOOK_DIR = os.getcwd()
ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

# Percorsi canonici (usati da tutto il notebook)
CLEANED_PATH = os.path.join(ROOT, "data", "processed", "database_cleaned.csv")
RAW_PATH = os.path.join(ROOT, "data", "raw", "car_sales.csv")                      # raw preferito del team
RAW_BACKUP_FROM_REPO = os.path.join(ROOT, "data", "raw", "car_sales_from_repo.csv")  # la tua copia proveniente da git
MAPPINGS_DIR = os.path.join(NOTEBOOK_DIR, "mappings")  # se i mappings sono in notebook/mappings
OUT_DIR = os.path.join(ROOT, "data", "processed")

# Controlli rapidi
print("working dir:", NOTEBOOK_DIR)
print("root:", ROOT)
print("CLEANED_PATH exists?", os.path.exists(CLEANED_PATH))
print("RAW_PATH exists?", os.path.exists(RAW_PATH))
print("RAW_BACKUP_FROM_REPO exists?", os.path.exists(RAW_BACKUP_FROM_REPO))
print("MAPPINGS_DIR exists?", os.path.exists(MAPPINGS_DIR))

if not os.path.exists(CLEANED_PATH):
    raise FileNotFoundError(f"File non trovato: {CLEANED_PATH}. Controlla che il file sia in data/processed/")

cols0 = pd.read_csv(CLEANED_PATH, nrows=0).columns.tolist()
parse_arg = ['Date'] if 'Date' in cols0 else None
df = pd.read_csv(CLEANED_PATH, parse_dates=parse_arg, low_memory=False)

print("Loaded df shape:", df.shape)

working dir: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/notebook
root: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale
CLEANED_PATH exists? True
RAW_PATH exists? True
RAW_BACKUP_FROM_REPO exists? True
MAPPINGS_DIR exists? True
Loaded df shape: (23906, 13)


In [14]:
CLEANED_PATH = "../data/processed/database_cleaned.csv"

import os
import pandas as pd

if not os.path.exists(CLEANED_PATH):
    raise FileNotFoundError(f"{CLEANED_PATH} non trovato. Controlla il percorso relativo.")

cols0 = pd.read_csv(CLEANED_PATH, nrows=0).columns.tolist()
parse_arg = ['Date'] if 'Date' in cols0 else None
df = pd.read_csv(CLEANED_PATH, parse_dates=parse_arg, low_memory=False)

print("Caricato:", CLEANED_PATH)
print("Shape:", df.shape)
print("Colonne:", df.columns.tolist())
display(df.head(5))

Caricato: ../data/processed/database_cleaned.csv
Shape: (23906, 13)
Colonne: ['Date', 'Customer Name', 'Gender', 'Annual Income', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Price ($)', 'Body Style', 'Dealer_Region']


,Date,Customer Name,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price ($),Body Style,Dealer_Region
0,2022-01-02,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,26000,SUV,Middletown
1,2022-01-02,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,19000,SUV,Aurora
2,2022-01-02,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500,Passenger,Greenville
3,2022-01-02,Giselle,Male,13500,Chrysler of Tri-Cities,Toyota,Celica,Overhead Camshaft,Manual,Pale White,14000,SUV,Pasco
4,2022-01-02,Grace,Male,1465000,Chrysler Plymouth,Acura,TL,DoubleÂ Overhead Camshaft,Auto,Red,24500,Hatchback,Janesville


## Generazione template mapping

Questa cella mostra i top values per le categoriche e crea `mappings/<col>_mapping_template.csv`.  
Compilare `canonical` nei template (lowercase, no leading/trailing spaces) e salvare come `mappings/<col>_mapping.csv`.  


In [ ]:
# Cell A - genera top values e template mapping
import os

MAPPINGS_DIR = "mappings"
os.makedirs(MAPPINGS_DIR, exist_ok=True)

cat_cols = ['Company', 'Dealer_Name', 'Model', 'Transmission', 'Gender', 'Customer Name']

for c in cat_cols:
    if c in df.columns:
        print(f"\n--- {c} (top 20) ---")
        display(df[c].astype(str).str.strip().str.lower().value_counts().head(20))
    else:
        print(f"\nColonna non presente: {c}")

# Generazione template mapping (raw -> canonical vuoto)
for col in cat_cols:
    if col in df.columns:
        vals = pd.Series(df[col].astype(str).str.strip().str.lower().unique())
        mapping_df = pd.DataFrame({'raw': vals, 'canonical': [''] * len(vals)})
        mapping_path = os.path.join(MAPPINGS_DIR, f"{col}_mapping_template.csv")
        mapping_df.sort_values('raw').to_csv(mapping_path, index=False)
        print("Template generato:", mapping_path)

##  Inizializzare i mapping (crea file `mappings/<col>_mapping.csv` se mancanti)

Scopo: creare file `mappings/<col>_mapping.csv` che contengono due colonne `raw` e `canonical`.  
Comportamento: se il file finale non esiste, viene scritto copiando i valori `raw` (puliti) come `canonical` di default. Non sovrascrive mapping già presenti.

In [ ]:
# crea mapping finali se non esistono (canonical = raw di default)
import os
import pandas as pd

if 'df' not in globals():
    raise RuntimeError("DataFrame `df` non trovato in memoria. Esegui prima la cella di caricamento.")

MAPPINGS_DIR = "mappings"
os.makedirs(MAPPINGS_DIR, exist_ok=True)

cat_cols = ['Company', 'Dealer_Name', 'Model', 'Transmission', 'Gender', 'Customer Name']

created = []
skipped = []
for col in cat_cols:
    template_path = os.path.join(MAPPINGS_DIR, f"{col}_mapping_template.csv")
    final_path = os.path.join(MAPPINGS_DIR, f"{col}_mapping.csv")
    if os.path.exists(final_path):
        skipped.append(final_path)
        continue
    # se esiste il template, usalo; altrimenti genera dai valori unici
    if os.path.exists(template_path):
        df_map = pd.read_csv(template_path, dtype=str).fillna('')
        df_map['raw'] = df_map['raw'].astype(str).str.strip().str.lower()
        df_map['canonical'] = df_map['raw']
    else:
        if col in df.columns:
            vals = pd.Series(df[col].astype(str).str.strip().str.lower().unique())
            df_map = pd.DataFrame({'raw': vals, 'canonical': vals})
        else:
            continue
    df_map = df_map[['raw','canonical']]
    df_map.to_csv(final_path, index=False)
    created.append(final_path)

print("Mapping creati:", created)
print("Mapping saltati (già esistenti):", skipped)

## Ispezione dei mapping creati

Obiettivo: verificare rapidamente i file `mappings/*_mapping.csv` generati automaticamente e identificare le correzioni prioritarie (top values, abbreviazioni, simboli, placeholder).  
Azione richiesta: aprire e modificare i file `mappings/Company_mapping.csv` e `mappings/Model_mapping.csv` per le voci più frequenti; per le altre colonne intervenire solo su eccezioni evidenti.

In [ ]:
# Anteprima dei mapping creati e conteggi raw per priorità
import os
import pandas as pd

MAPPINGS_DIR = "mappings"
cat_cols = ['Company', 'Dealer_Name', 'Model', 'Transmission', 'Gender', 'Customer Name']

for col in cat_cols:
    path = os.path.join(MAPPINGS_DIR, f"{col}_mapping.csv")
    if os.path.exists(path):
        m = pd.read_csv(path, dtype=str).fillna('')
        print(f"\n-- {col} mapping -- {path}")
        display(m.head(10))
        print(f"Total unique raw values: {len(m)}")
    else:
        print(f"\n{col}: mapping file non trovato: {path}")

## Ispezione avanzata: individuare candidate per canonicalizzazione

Obiettivo: identificare le voci che probabilmente richiedono un'unificazione (abbreviazioni, punteggiatura, varianti corte, typo evidenti).  
Eseguire la cella di codice per ottenere:
- conteggio valori Company ordinato;
- eventuali valori Company contenenti caratteri speciali, abbreviazioni o molto corti;
- top-models sospetti (es. doppioni simili).

In [ ]:
# Ispezione avanzata per Company e Model
import pandas as pd

# company counts
company_counts = df['Company'].astype(str).str.strip().str.lower().value_counts()
print("Company - top (count):")
display(company_counts.head(50))

# candidate con simboli o short codes
candidates_symbols = [v for v in company_counts.index if any(ch in v for ch in ".-/&()'") or len(v) <= 3]
print("\nCompany - candidate contenenti simboli o short (esempi):")
display(candidates_symbols[:50])

# valori molto rari (f < 0.1%) - per valutare eventuale "other"
freq = company_counts / len(df)
rare_companies = freq[freq < 0.001]  # soglia 0.1% (adatta nella tua analisi)
print("\nCompany molto rare (f < 0.1%):")
display(rare_companies)

# per Model: mostra similarità semplicistica (possibili duplicati basati su trim)
model_counts = df['Model'].astype(str).str.strip().str.lower().value_counts()
print("\nModel - top (count):")
display(model_counts.head(50))

# mostra esempi di modelli simili per manual check (esempio: same prefix)
print("\nEsempi modelli che contengono 'passat' o 'jetta' per verificarne varianti:")
display(model_counts[model_counts.index.str.contains('passat')].head(50))
display(model_counts[model_counts.index.str.contains('jetta')].head(50))

## Correzioni consigliate per `Company`

Obiettivo: correggere le poche varianti evidenti per unificare i brand.  
Suggerimento immediato: mappare `mercedes-b` → `mercedes-benz`.  
Nota: `bmw` è valido come canonical (non va cambiato).

In [ ]:
# backup e applicazione correzioni mirate su Company_mapping.csv
import os
import pandas as pd
from shutil import copyfile

MAPPINGS_DIR = "mappings"
company_map_path = os.path.join(MAPPINGS_DIR, "Company_mapping.csv")
backup_path = os.path.join(MAPPINGS_DIR, "Company_mapping_backup.csv")

if not os.path.exists(company_map_path):
    raise FileNotFoundError(f"{company_map_path} non trovato. Assicurati che sia stato creato.")

# 1) fai un backup ì
if not os.path.exists(backup_path):
    copyfile(company_map_path, backup_path)
    print("Backup creato:", backup_path)
else:
    print("Backup già esistente:", backup_path)

# 2) correzioni proposte
corrections = {
    "mercedes-b": "mercedes-benz",
}

# 3) carica, applica correzioni su canonical, salva
df_map = pd.read_csv(company_map_path, dtype=str).fillna('')
df_map['raw'] = df_map['raw'].astype(str).str.strip().str.lower()
# solo per sicurezza: mantieni canonical già presente se non vuoto, altrimenti sostituisci
df_map['canonical'] = df_map['canonical'].astype(str).str.strip().str.lower()

# Applica le correzioni: sovrascrive canonical per le raw presenti in corrections
applied = []
for raw_val, new_can in corrections.items():
    mask = df_map['raw'] == raw_val
    if mask.any():
        df_map.loc[mask, 'canonical'] = new_can
        applied.append(raw_val)

df_map.to_csv(company_map_path, index=False)
print("Correzioni applicate per le raw:", applied)
print("Se vuoi aggiungere altre correzioni, aggiorna il dizionario 'corrections' e riesegui.")
display(df_map[df_map['raw'].isin(applied)].head(20))

## Nota: aggiornamento mapping Company

Ho creato un backup (`mappings/Company_mapping_backup.csv`) e applicato la correzione mirata:
- `mercedes-b` → `mercedes-benz`

Questa modifica è stata salvata in `mappings/Company_mapping.csv`. 

In [ ]:
# Riapplica mapping aggiornati 
import os
import pandas as pd

MAPPINGS_DIR = "mappings"
cat_cols = ['Company', 'Dealer_Name', 'Model', 'Transmission', 'Gender', 'Customer Name']

def load_mapping_dict(path):
    m = pd.read_csv(path, dtype=str).fillna('')
    m['raw'] = m['raw'].astype(str).str.strip().str.lower()
    m['canonical'] = m['canonical'].astype(str).str.strip().str.lower()
    return {r: c for r, c in zip(m['raw'], m['canonical']) if c != ''}

for c in cat_cols:
    if c in df.columns:
        mapping_file = os.path.join(MAPPINGS_DIR, f"{c}_mapping.csv")
        canonical_col = c.lower().replace(' ', '_') + '_canonical'
        if os.path.exists(mapping_file):
            mapping_dict = load_mapping_dict(mapping_file)
            df[canonical_col] = df[c].astype(str).str.strip().str.lower().map(lambda x: mapping_dict.get(x, x) if pd.notna(x) else None)
            print("Mapping applicato:", c)
        else:
            df[canonical_col] = df[c].astype(str).str.strip().str.lower().replace({'nan': None})
            print("Fallback (no mapping):", c)

# Verifica rapida di presenza colonna
print("\nColonne create/aggiornate:")
print([c for c in df.columns if c.endswith('_canonical')])

## QA post-mapping e salvataggio sample

Eseguire la cella di controllo per verificare distinct prima/dopo, visualizzare i top canonical e salvare un sample per la condivisione interna/presentazione.

In [ ]:
# QA e salvataggio sample
def no_surrounding_spaces(series):
    vals = [x for x in series.dropna().unique() if isinstance(x, str)]
    return all(v == v.strip() for v in vals)

for c in ['Company','Model']:
    if c in df.columns:
        raw_n = df[c].nunique()
        can_col = c.lower().replace(' ', '_') + '_canonical'
        can_n = df[can_col].nunique() if can_col in df.columns else None
        print(f"\n{c}: raw distinct = {raw_n}, canonical distinct = {can_n}")
        if can_col in df.columns:
            display(df[can_col].value_counts().head(30))
            print("No surrounding spaces:", no_surrounding_spaces(df[can_col]))

# salva sample per condivisione (non committare dati grezzi)
sample = df.sample(n=min(200, len(df)), random_state=42)
sample_path = os.path.join("..","data","processed","sample_standardized.csv") if os.path.exists(os.path.join("..","data","processed")) else os.path.join("data","processed","sample_standardized.csv")
os.makedirs(os.path.dirname(sample_path), exist_ok=True)
sample.to_csv(sample_path, index=False)
print("\nSample salvato in:", sample_path)

In [ ]:
# Cell: prepara city_state e salva dataset dettaglio per Tableau
import pandas as pd
import os

CLEANED_PATH = "../data/processed/database_cleaned.csv"  # controlla che questo sia il file giusto
OUT_DETAIL = "../data/processed/database_for_tableau_city_state.csv"

if 'df' not in globals():
    df = pd.read_csv(CLEANED_PATH, low_memory=False)

# normalizzazione conservativa: Title Case e trim
df['dealer_region_mapped'] = df['Dealer_Region'].astype(str).str.strip().str.title()

# city_state: se già contiene una virgola mantieni, altrimenti aggiungi ", USA"
def mk_city_state(v):
    s = str(v).strip()
    if s == "" or s.lower() in ['nan','none']:
        return ""
    return s if ',' in s else f"{s}, USA"

df['city_state'] = df['dealer_region_mapped'].apply(mk_city_state)
# salva dettaglio
os.makedirs(os.path.dirname(OUT_DETAIL), exist_ok=True)
df.to_csv(OUT_DETAIL, index=False)
print("Salvato dettaglio per Tableau:", OUT_DETAIL)
display(df[['dealer_region_mapped','city_state']].drop_duplicates().head(20))

Salvato dettaglio per Tableau: ../data/processed/database_for_tableau_city_state.csv


,dealer_region_mapped,city_state
0,Middletown,"Middletown, USA"
1,Aurora,"Aurora, USA"
2,Greenville,"Greenville, USA"
3,Pasco,"Pasco, USA"
4,Janesville,"Janesville, USA"
5,Scottsdale,"Scottsdale, USA"
6,Austin,"Austin, USA"
